In [1]:
import numpy as np
import csv
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
import dash
from dash import Dash, dcc, html, Input, Output, callback, dash_table, State, MATCH, ALL, Patch, State, ctx, register_page, page_container, no_update
import dash_bootstrap_components as dbc
import time
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import pymysql
import dash_daq as daq

In [2]:
app = dash. Dash(__name__,  title='Добро пожаловать в робоадвайзер!', suppress_callback_exceptions=True)

In [3]:
MAIN_BLOCK = {
    "position": "fixed", 
    "top": '15rem', 
    "left": "calc(50% - 30rem)", # Вычисление левого отступа
    "height": "35%",
    "bottom": 0,
    "width": '60rem', 
    "padding": "1rem 2rem", 
    "background-color": "#FFFEFE",
    "border-top-left-radius": "20px",
    "border-bottom-left-radius": "20px",
    "border-top-right-radius": "20px",
    "border-bottom-right-radius": "20px",
}


In [4]:
app.layout = html.Div([
                        html.Div([
                                        html.H1(children = 'Добро пожаловать в Робоадвайзер!',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),
                                        html.H3(children = 'Робоадвайзер — это автоматизированный сервис, который помогает вам управлять инвестициями на основе ваших финансовых \
                                                            целей и предпочтений. Используя алгоритмы и данные, он подберет оптимальный портфель активов, чтобы максимально эффективно \
                                                            управлять вашими вложениями.',
                                                style = {
                                                         'textAlign': 'center',  'color': '#B2B3B5', 'font': 'Uni Neue', 'font-size': 16, 'font-weight': '1', 
                                                        }
                                                ),
                                        html.Div([ 
                                                    dbc.Card(
                                                            [
                                                                dbc.CardImg(src="https://cdn4.iconfinder.com/data/icons/business-1-3/512/briefcase-1024.png", top=True, 
                                                                            style={
                                                                                     'margin': '0 auto',
                                                                                     'display': 'flex',
                                                                                     'justify-content': 'center',
                                                                                     "width": 35, "height": 35
                                                                                    }
                                                                           ),
                                                                dbc.CardBody(
                                                                                dbc.Button('Рассчитать портфель',
                                                                                            href = "http://localhost:1110"),
                                                                                             style = {
                                                                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                                                                     }
                                                                                            ),
                                                            ],
                                                            style={ "width": 100, 
                                                                    "height": 100, 
                                                                    'border': '2px solid #C1A068',  # Круглый контур с тем же цветом, что и фон
                                                                    'border-radius': '20px',  # Радиус скругления для круглой формы
                                                                    'padding': '10px 20px',  # Отступы внутри кнопки
                                                                    'cursor': 'pointer',  # Изменение курсора при наведении
                                                                  }
                                                        ),
                                                 ],
                                                    style={
                                                                'position': 'absolute',  # Позиционируем блок относительно родительского контейнера
                                                                'left': '50%',  # Выравниваем по горизонтали
                                                                'transform': 'translateX(-50%)',  # Центрируем по горизонтали
                                                                'bottom': '10%',  # Прижимаем к нижней границе родительского контейнера
                                                                'text-align': 'center',
                                                            }),
                                        html.Div([
                                                    dbc.Card(
                                                                [
                                                                    dbc.CardImg(src="https://cdn3.iconfinder.com/data/icons/user-icon-3-1/100/user_3_Artboard_1_copy_2-256.png", top=True, 
                                                                                style={
                                                                                         'margin': '0 auto',
                                                                                         'display': 'flex',
                                                                                         'justify-content': 'center',
                                                                                         "width": 35, "height": 35
                                                                                        }
                                                                               ),
                                                                    dbc.CardBody(
                                                                                    dbc.Button('Выбрать профиль',
                                                                                                href = "http://localhost:1113/"),
                                                                                                 style = {
                                                                                                             'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                                                                         }
                                                                                                ),
                                                                ],
                                                                style={ "width": 100, 
                                                                        "height": 100, 
                                                                        'border': '2px solid #C1A068',  # Круглый контур с тем же цветом, что и фон
                                                                        'border-radius': '20px',  # Радиус скругления для круглой формы
                                                                        'padding': '10px 20px',  # Отступы внутри кнопки
                                                                        'cursor': 'pointer',  # Изменение курсора при наведении
                                                                      }
                                                            ),
                                                ],
                                                    style={
                                                                'position': 'absolute',  # Позиционируем блок относительно родительского контейнера
                                                                'left': '25%',  # Выравниваем по горизонтали  
                                                                'bottom': '10%',  # Прижимаем к нижней границе родительского контейнера
                                                                'text-align': 'center',
                                                            }),
                                        html.Div([
                                                    dbc.Card(
                                                                [
                                                                    dbc.CardImg(src="https://habrastorage.org/webt/5b/81/b3/5b81b378e86e5556460500.png", top=True, 
                                                                                style={
                                                                                         'margin': '0 auto',
                                                                                         'display': 'flex',
                                                                                         'justify-content': 'center',
                                                                                         "width": 35, "height": 35
                                                                                        }
                                                                               ),
                                                                    dbc.CardBody(
                                                                                    dbc.Button('Справочник профилей',
                                                                                                href = "http://localhost:1115/"),
                                                                                                 style = {
                                                                                                             'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                                                                         }
                                                                                                ),
                                                                ],
                                                                style={ "width": 100, 
                                                                        "height": 100, 
                                                                        'border': '2px solid #C1A068',  # Круглый контур с тем же цветом, что и фон
                                                                        'border-radius': '20px',  # Радиус скругления для круглой формы
                                                                        'padding': '10px 20px',  # Отступы внутри кнопки
                                                                        'cursor': 'pointer',  # Изменение курсора при наведении
                                                                      }
                                                            ),
                                                ],
                                                    style={
                                                                'position': 'absolute',  # Позиционируем блок относительно родительского контейнера
                                                                'left': '61%',  # Выравниваем по горизонтали  
                                                                'bottom': '10%',  # Прижимаем к нижней границе родительского контейнера
                                                                'text-align': 'center',
                                                            }),
        
                                ], style = MAIN_BLOCK),
                
    
                      
    dash.page_container],      
                        ### Картинка на фон страницы
                        style={'background-image': 'url(https://kartinki.pics/uploads/posts/2022-12/1671725866_kartinkin-net-p-neboskrebi-kartinki-krasivo-30.jpg)',
                               'background-size': '110%',
                               'left': 0,
                               "top": 0, 
                               'width': '110vw',
                               'height': '110vh',
                               "position": "fixed", 
                              }
                     )
if __name__ == '__main__':
    app.run_server(port=1111, host="0.0.0.0")